Use the following Azure Databricks storage setup block only if you are using Azure Databricks. You can refer to the instructions here to get started:
https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

If you are using Synapse Spark and if your data is residing on the storage attached to the Synapse Spark workspace, you can skip the below storage setup section.

In [ ]:
%scala
val storageAccountName = "<INSERT STORAGE ACCOUNT>"
val fileSystemName = "<INSERT CONTAINER NAME>"

val commonPath = "abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net"

# AAD Application Details
val appID = "<INSERT APP ID>"
val secret = "<INSERT SECRET>"
val tenantID = "<INSERT TENANT ID>"

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + secret + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

Install the Event Hub library in the spark cluster before proceeding to the next step.
Spark Event Hubs connector - com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.21 for Spark 3.x

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# connection string of Event Hubs Namespace
EHConnectionString = "<INSERT EVENT HUB CONN STR>"

EHConfig = {}
EHConfig['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(EHConnectionString)

EHStreamDF = spark.readStream.format("eventhubs").options(**EHConfig).load()

In [ ]:
print( EHStreamDF.isStreaming)
print( EHStreamDF.printSchema())

In [ ]:
JsonSchema = StructType() \
.add("tripId", StringType()) \
.add("createdAt", TimestampType()) \
.add("startLocation", StringType()) \
.add("endLocation", StringType()) \
.add("distance", IntegerType()) \
.add("fare", IntegerType())

In [ ]:
stringDF=EHStreamDF.selectExpr("CAST(body AS STRING)")

In [ ]:
jsonDF=stringDF.withColumn('tripjson', from_json(col('body'),schema=JsonSchema))

In [ ]:
EHStreamJsonDF=jsonDF.select("tripjson.*")
display(EHStreamJsonDF)

In [ ]:
EHStreamJsonDF.selectExpr(
                  "tripId"\
                  ,"createdAt"\
                  ,"startLocation"\
                  ,"endLocation"\
                  ,"distance"\
                  ,"fare")\
.writeStream.format("delta")\
.outputMode("append")\
.option("checkpointLocation", "dbfs:/tripsCheckpointLocation1/")\
.option("mergeSchema", "true")\
.start("dbfs:/TripsEventHubDelta1")

In [ ]:
EHStreamJsonDF.groupBy(window('createdAt',"1 minutes"),'startLocation').count().orderBy('window') \
.writeStream.format("delta") \
.outputMode("complete") \
.option("truncate", "false") \
.option("checkpointLocation", "dbfs:/tripsCheckpointLocationTumbling2/") \
.option("mergeSchema", "true") \
.start("dbfs:/TripsEventHubDeltaTumbling2") 

In [ ]:
EHStreamJsonDF.groupBy(window('createdAt',"1 minutes"),'startLocation').count().orderBy('window')\
.writeStream.format("memory") \
.outputMode("complete") \
.option("truncate", "false") \
.option("checkpointLocation", "dbfs:/tripsCheckpointLocation/") \
.queryName("TripsTumblingQuery").start() 

In [ ]:
%sql
SELECT * FROM TripsTumblingQuery ORDER BY Window desc

In [ ]:
%sql
DROP TABLE IF EXISTS TripsAggTumbling;
CREATE TABLE IF NOT EXISTS TripsAggTumbling
USING DELTA
LOCATION "dbfs:/TripsEventHubDeltaTumbling2/"

In [ ]:
%sql
SELECT * FROM TripsAggTumbling ORDER BY Window desc

In [ ]:
for s in spark.streams.active:
    s.stop()